In [1]:
pip install ultralytics


  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl.metadata (59 kB)
  Using cached matplotlib-3.9.4-cp39-cp39-win_amd64.whl.metadata (11 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached pillow-11.1.0-cp39-cp39-win_amd64.whl.metadata (9.3 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl.metadata (60 kB)
  Using cached torch-2.6.0-cp39-cp39-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp39-cp39-win_amd64.whl.metadata (6.3 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached pandas-2.2.3-cp39-cp39-win_amd64.whl.metadata (19 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached contourpy-1.3.0-cp39-cp39-win_amd64.whl.metadata (5.4 kB)
  Using cached c

In [2]:
pip install opencv-python numpy filterpy

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110599 sha256=1d97280ffed610229505790a9112b7a264c274281b7e1302a9ceb22d5e0d4153
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\53\e6\de\a09ea01e923aaf88b9f8c7c44329e857b2c1a31901167e55e6
Successfully built filterpy
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from filterpy.kalman import KalmanFilter
import math

def create_kalman_filter(x_center, y_center):
    kf = KalmanFilter(dim_x=4, dim_z=2)
    kf.x = np.array([[x_center], [y_center], [0], [0]])
    kf.F = np.array([[1, 0, 1, 0],
                     [0, 1, 0, 1],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])
    kf.H = np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0]])
    kf.P *= np.eye(4) * 1000
    kf.R = np.array([[10, 0], [0, 10]])
    kf.Q *= np.eye(4) * 0.1
    return kf
    

def predict_future_positions(kf, steps):
    future_positions = []
    state = kf.x.copy()
    for _ in range(steps):
        state = np.dot(kf.F, state)
        future_positions.append((int(state[0]), int(state[1])))
    return future_positions

def calculate_distance(point1, point2):
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

model_path = 'runs/detect/train/weights/best.pt'
model = YOLO(model_path)

video_path = 'test13.mp4'
cap = cv2.VideoCapture(video_path)

kalman_filters = {}
object_states = {}
max_distance = 22  # Максимальное расстояние для сопоставления дронов
max_time_gap = 5   # Максимальное время (в кадрах), чтобы считать дрон тем же

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_tracking_video.mp4', fourcc, fps, (width, height))

frame_number = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(source=frame, conf=0.65, iou=0.3)
    
    for result in results:
        for box in result.boxes.data:
            x1, y1, x2, y2, conf, cls = box.tolist()
            x_center = (x1 + x2) / 2
            y_center = (y1 + y2) / 2

            class_name = model.names[int(cls)]
            current_position = (x_center, y_center)

            # Проверка на существование дронов
            matched_id = None
            for obj_id, state in object_states.items():
                last_position = state['coordinates'][-1] if state['coordinates'] else None
                if last_position:
                    distance = calculate_distance(current_position, last_position)
                    time_gap = frame_number - state['last_seen']
                    if distance < max_distance and time_gap < max_time_gap:
                        matched_id = obj_id
                        break

            if matched_id is not None:
                # Обновление существующего дрона
                kf = kalman_filters[matched_id]
                kf.predict()
                kf.update([x_center, y_center])
                x_pred, y_pred = kf.x[:2]
                object_states[matched_id]['last_seen'] = frame_number
                if len(object_states[matched_id]['coordinates']) > 25:
                    object_states[matched_id]['coordinates'].pop(0)
                    object_states[matched_id]['coordinates'].append((int(x_pred), int(y_pred)))
                else:
                    object_states[matched_id]['coordinates'].append((int(x_pred), int(y_pred)))
            else:
                # Создание нового отслеживаемого дрона
                matched_id = f"{class_name}_{frame_number}_{int(x_center)}_{int(y_center)}"
                kalman_filters[matched_id] = create_kalman_filter(x_center, y_center)
                kalman_filters[matched_id].predict()
                kalman_filters[matched_id].update([x_center, y_center])
                x_pred, y_pred = kalman_filters[matched_id].x[:2]
                object_states[matched_id] = {'last_seen': frame_number, 'coordinates': [(int(x_pred), int(y_pred))]}

            # Отрисовка прямоугольника и текста
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            text = f"{class_name}: {conf:.2f}"
            cv2.putText(frame, text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (128, 0, 128), 1)

            # Отрисовка пройденного пути
            traveled_path_coordinates = object_states[matched_id]['coordinates']
            for i in range(len(traveled_path_coordinates) - 1):
                cv2.line(frame, traveled_path_coordinates[i], traveled_path_coordinates[i + 1], (0, 0, 255), 2)

            # Предсказание будущих позиций
            future_positions = predict_future_positions(kalman_filters[matched_id], steps=22)
            for i in range(len(future_positions) - 1):
                cv2.line(frame, future_positions[i], future_positions[i + 1], (255, 0, 0), 2)

            # Удаление старых объектов
            if len(kalman_filters) > 22 and len(object_states) > 22:
                # Удаляем самый старый элемент по ключу
                oldest_key = next(iter(kalman_filters))  # Получаем первый ключ
                del kalman_filters[oldest_key]
                del object_states[oldest_key]

    frame_number += 1
    out.write(frame)
    cv2.imshow("Drone Detection & Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()